# Model Evaluation Final - Airbnb Reviews

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, roc_curve, roc_auc_score, precision_recall_curve, precision_score, recall_score, average_precision_score

%matplotlib inline
plt.rcParams.update({"font.size": 14})

## Execution Time Comparison

In [ ]:
# All thread
# 32.9 s ± 2.7 s per loop (mean ± std. dev. of 3 runs, 1 loop each)
# 2.54 s ± 6.02 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
# 5.41 s ± 105 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
# 6min 20s ± 3.26 s per loop (mean ± std. dev. of 3 runs, 1 loop each)
# 4min 9s ± 2.65 s per loop (mean ± std. dev. of 3 runs, 1 loop each)

# Single thread
# 37.9 s ± 1.71 s per loop (mean ± std. dev. of 3 runs, 1 loop each)
# 3.83 s ± 38.2 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
# 5.73 s ± 79.3 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
# 6min 19s ± 5.08 s per loop (mean ± std. dev. of 3 runs, 1 loop each)
# 4min 22s ± 6.14 s per loop (mean ± std. dev. of 3 runs, 1 loop each)

In [ ]:
labels = ['LOF', 'DBSCAN', 'iForest', 'OCSVM', 'SUOD']
x = np.arange(len(labels))

# Draw the plot
fig, ax = plt.subplots(figsize=(8,5))
width = 0.3

mean = [32.9, 2.54, 5.41, 380, 249]
std = [2.7, 0.6, 0.12, 3.26, 2.65]
rects1 = ax.bar(x, mean, yerr=std, width=0.3, align='center', alpha=0.8, ecolor='black', capsize=10)

mean = [37.9, 3.83, 5.73, 379, 262]
std = [1.71, 0.38, 0.79, 5.08, 6.14]
rects2 = ax.bar(x+width, mean, yerr=std, width=0.3, align='center', alpha=0.8, ecolor='black', capsize=10)

ax.set_xlabel('\nTuned Outlier Detection Models')
ax.set_ylabel('Execution Time in Seconds')
ax.set_xticks(x+0.15)
ax.set_xticklabels(labels)
ax.set_title('Execution Time Comparison\n')
ax.legend( (rects1[0], rects2[0]), ('n_jobs=-1', 'n_jobs=1') )
ax.yaxis.grid(True)

plt.tight_layout()
plt.savefig('figures/execution_time_comparison_opt.png', bbox_inches="tight")
plt.show()

## Confusion Matrix

In [ ]:
filename = "reviews_sample_labelled.csv"
df_label = pd.read_csv(filename, sep=";")
df_label.shape

In [ ]:
filename = "reviews_outlier_predtions_final.csv"
df_pred = pd.read_csv(filename, sep=";")
df_pred.shape

In [ ]:
def draw_confusion_matrix(model, name):
    y_true = df_label["label"]
    y_pred = df_pred[model]
    cf_matrix = confusion_matrix(y_true, y_pred)

    group_names = ['True Neg','False Pos','False Neg','True Pos']
    group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)

    ax = sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')

    ax.set_title(f'Confusion Matrix for Tuned {name}\n');
    ax.set_xlabel('\nPredicted Values')
    ax.set_ylabel('Actual Values ');

    ax.xaxis.set_ticklabels(['Inlier','Outlier'])
    ax.yaxis.set_ticklabels(['Inlier','Outlier'])
    plt.savefig(f'figures/confusion_matrix_tuned_{model}.png', bbox_inches="tight")
    plt.show()
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    print(f"Model: {name} => Precision: {precision:.2f}, Recall: {recall:.2f}, F1: {f1:.2f}")

### LOF

In [ ]:
draw_confusion_matrix("lof", "LOF")

### DBSCAN

In [ ]:
draw_confusion_matrix("dbscan", "DBSCAN")

### iForest

In [ ]:
draw_confusion_matrix("iforest", "iForest")

### OCSVM

In [ ]:
draw_confusion_matrix("ocsvm", "OCSVM")

### SUOD

In [ ]:
draw_confusion_matrix("suod", "SUOD")

## ROC AUC Score

In [ ]:
sample_length = len(df_label["label"])

lof_auc = roc_auc_score(df_label["label"], df_pred["lof_score"])
dbscan_auc = roc_auc_score(df_label["label"], df_pred["dbscan"])
iforest_auc = roc_auc_score(df_label["label"], df_pred["iforest_score"])
ocsvm_auc = roc_auc_score(df_label["label"], df_pred["ocsvm_score"])
suod_auc = roc_auc_score(df_label["label"], df_pred["suod_score"])

# summarize scores
print('LOF => ROC AUC=%.3f' % (lof_auc))
print('DBSCAN => ROC AUC=%.3f' % (dbscan_auc))
print('iForest => ROC AUC=%.3f' % (iforest_auc))
print('OCSVM => ROC AUC=%.3f' % (ocsvm_auc))
print('SUOD => ROC AUC=%.3f' % (suod_auc))

# calculate roc curves
random_fpr, random_tpr, _ = roc_curve(df_label["label"], [0]*sample_length)
lof_fpr, lof_tpr, _ = roc_curve(df_label["label"], df_pred["lof_score"])
dbscan_fpr, dbscan_tpr, _ = roc_curve(df_label["label"], df_pred["dbscan"])
iforest_fpr, iforest_tpr, _ = roc_curve(df_label["label"], df_pred["iforest_score"])
ocsvm_fpr, ocsvm_tpr, _ = roc_curve(df_label["label"], df_pred["ocsvm_score"])
suod_fpr, suod_tpr, _ = roc_curve(df_label["label"], df_pred["suod_score"])

plt.figure(figsize=(10,6))
# plot the roc curve for the model
plt.plot(random_fpr, random_tpr, linestyle='--')
plt.plot(lof_fpr, lof_tpr, label=f'LOF')
plt.plot(dbscan_fpr, dbscan_tpr, label=f'DBSCAN')
plt.plot(iforest_fpr, iforest_tpr, label=f'iForest')
plt.plot(ocsvm_fpr, ocsvm_tpr, label=f'OCSVM')
plt.plot(suod_fpr, suod_tpr, label=f'SUOD')

# axis labels
plt.xlabel('\nFalse Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve\n')
plt.legend()
plt.savefig(f'figures/roc_final.png', bbox_inches="tight")
plt.show()

## Precision Recall Curve

In [ ]:
lof_auc = average_precision_score(df_label["label"], df_pred["lof_score"])
dbscan_auc = average_precision_score(df_label["label"], df_pred["dbscan"])
iforest_auc = average_precision_score(df_label["label"], df_pred["iforest_score"])
ocsvm_auc = average_precision_score(df_label["label"], df_pred["ocsvm_score"])
suod_auc = average_precision_score(df_label["label"], df_pred["suod_score"])

# summarize scores
print('LOF => PR AUC=%.3f' % (lof_auc))
print('DBSCAN => PR AUC=%.3f' % (dbscan_auc))
print('iForest => PR AUC=%.3f' % (iforest_auc))
print('OCSVM => PR AUC=%.3f' % (ocsvm_auc))
print('SUOD => PR AUC=%.3f' % (suod_auc))

precision_lof, recall_lof, th_lof = precision_recall_curve(df_label["label"], df_pred["lof_score"])
precision_dbscan, recall_dbscan, th_dbscan = precision_recall_curve(df_label["label"], df_pred["dbscan"])
precision_iforest, recall_iforest, th_iforest = precision_recall_curve(df_label["label"], df_pred["iforest_score"])
precision_ocsvm, recall_ocsvm, th_ocsvm = precision_recall_curve(df_label["label"], df_pred["ocsvm_score"])
precision_suod, recall_suod, th_suod = precision_recall_curve(df_label["label"], df_pred["suod_score"])

plt.figure(figsize=(10,6))
plt.plot([0,1], [0.5,0.5], linestyle='--')
plt.plot(recall_lof, precision_lof, label='LOF')
plt.plot(recall_dbscan, precision_dbscan, label='DBSCAN')
plt.plot(recall_iforest, precision_iforest, label='iForest')
plt.plot(recall_ocsvm, precision_ocsvm, label='OCSVM')
plt.plot(recall_suod, precision_suod, label='SUOD')

plt.title('Precision-Recall Curve\n')
plt.ylabel('Precision')
plt.xlabel('\nRecall')
plt.legend()
plt.savefig(f'figures/precision_recall_curve_final.png')
plt.show()

In [ ]:
# Draw precision and recall plot
def draw_precision_recall_plot(th, precision, recall, model):
    plt.figure(figsize=(10,6))
    plt.plot(th, precision[:-1], label="Precision")
    plt.plot(th, recall[:-1], label="Recall")
    plt.title(f"Precision-Recall vs Threshold Plot for Tuned {model}\n")
    plt.xlabel("\nThreshold")
    plt.ylabel("Value")
    plt.legend()
#     plt.grid()
    plt.savefig(f'figures/threshold_{model.lower()}_tuned.png')
    plt.draw()

In [ ]:
draw_precision_recall_plot(th_lof, precision_lof, recall_lof, "LOF")

In [ ]:
draw_precision_recall_plot(th_iforest, precision_iforest, recall_iforest, "iForest")

In [ ]:
draw_precision_recall_plot(th_ocsvm, precision_ocsvm, recall_ocsvm, "OCSVM")

In [ ]:
draw_precision_recall_plot(th_suod, precision_suod, recall_suod, "SUOD")